In [ ]:
# conda install -c conda-forge gdk-pixbuf
# pip install pango
# pip install pycairo

######## ALTERNATIVE with annoying watermark
# pip install aspose-words

In [88]:
import os
import glob
import json
import ast
# import md2pdf

from openai import OpenAI

def load_api_key(config_file):
    with open(config_file) as f:
        config = json.load(f)
    client = OpenAI(api_key=config['api_key'])
    return client

def get_text_files(directory):
    list_of_files = glob.glob(os.path.join(directory, '*.txt'))
    latest_file, file_name = [""," "]
    if len(list_of_files) > 0:
        latest_file = max(list_of_files, key=os.path.getctime)
        full_name = os.path.basename(latest_file)
        file_name = os.path.splitext(full_name)
    return latest_file, file_name[0]

def read_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    return content

def output_directories(path,name,extension):
    if not os.path.exists(path):
        os.makedirs(path)
    n_files = ""
    # n_files = len(get_text_files(path))
    # n_files = n_files + 1
    # if n_files < 10:
    #     n_files = "0" + str(n_files)
    # else:
    #     n_files = str(n_files)
    nameFile = name + n_files + extension
    return(path + nameFile)

# from __future__ import print_function

def find_values(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

directory = "./mealPlans/"
config_file = "config.json"

client = load_api_key(config_file)


In [84]:

def get_completion(prompt, client, model="gpt-4", temperature=0, ):

    messages = [{"role": "system", "content": 'You are a inventory expert in a grocery store'},
                {"role": "user", "content": prompt}]
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    # response = completion.model_dump_json(indent=2)
    return completion.choices[0].message.content # message["content"]



def get_completion_from_messages(messages, client, model="gpt-4", temperature=0.3):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]

In [82]:
###################################
####### Ingredients  ############
###################################
# INPUT: Lists of Ingredients for each day in Danish.
# OUTPUT: English translation of the lists of Ingredients for each day.

# Setup new file of MealPlans

text_file, file_name = get_text_files(directory)

output_Ingredients_dir = "./outputIngredients/"
outputIngredientsFile = output_directories(output_Ingredients_dir, file_name + "_Ingredients_",".txt")

content = read_file(text_file)
prompt = f"""
Your task is to translate the  
lists of ingredients from Danish to English.\

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputIngredientsFile, "w")
f.write(response)
f.close()  
    

1
yes?
0
Thai soup with shrimp

INGREDIENTS

2 regular onions
2 bell peppers
3 cloves of garlic
2 tbsp oil for frying
2 tbsp curry or red curry paste
7 dl water
1 bouillon cube
400 ml coconut milk
1 tbsp paprika
1 tbsp sugar
chili powder to taste
1 dl rice
salt and pepper
200 g shrimp, thawed

One pot noodles with egg and vegetables

INGREDIENTS

2 cloves of garlic
1 bell pepper
1 bunch of spring onions
3 carrots
300 g cabbage white cabbage or pointed cabbage
1 tbsp oil
2 tsp ginger possibly freshly grated
250 g whole grain noodles
9 dl water
1 bouillon cube
4 eggs
salt and pepper
Sauce
3 tbsp lime juice
2 tbsp soy sauce
4 tsp tahini or peanut butter
2 tsp sugar
1 tsp garlic powder

Spicy chicken in a dish

INGREDIENTS

400 g chicken breast, thigh fillet or inner fillet
oil for frying
2 bell peppers
2 regular onions
2 cloves of garlic
1 can chopped tomatoes
1 pack buko spicy cheese
250 ml oma cooking cream 4%
2 tbsp paprika
1 bouillon cube
salt and pepper
Side dish
1 pack cauliflower r

In [87]:
###################################
####### ShoppingList  ############
###################################
# INPUT: Lists of Ingredients for each day in English.
# OUTPUT: Json object containing a table with the ingredients 
#   grouped by category and the quantities added up.

# Setup new file of ShoppingLists

text_file, file_name = get_text_files(output_Ingredients_dir)

output_ShoppingList_dir = "./outputShoppingList/"
outputShoppingListFile = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_",".json")

content = read_file(text_file)
prompt = f"""
Your task is to create a table with all the ingredients grouped
by category and quantity.\

The output table must be in Json format.\

Your response must only be a Json object and nothing else.\

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputShoppingListFile, "w")
f.write(response)
f.close()  
    

{
  "Vegetables": {
    "Onions": 6,
    "Bell Peppers": 7,
    "Garlic Cloves": 9,
    "Spring Onions": 1,
    "Carrots": 3,
    "Cabbage": 300,
    "Red Bell Peppers": 2
  },
  "Proteins": {
    "Shrimp": 200,
    "Eggs": 4,
    "Chicken Breast": 400,
    "Kidney Beans": 2,
    "Cod Fillets": 225
  },
  "Grains": {
    "Rice": 1.3,
    "Whole Grain Noodles": 250,
    "Whole Grain Tortillas": 6,
    "Bread Croutons": 1
  },
  "Dairy": {
    "Buko Spicy Cheese": 1,
    "Oma Cooking Cream 4%": 250,
    "Grated Cheddar": 75
  },
  "Canned Goods": {
    "Coconut Milk": 400,
    "Chopped Tomatoes": 2,
    "Lobster Soup": 750
  },
  "Spices and Condiments": {
    "Curry or Red Curry Paste": 2,
    "Paprika": 4,
    "Chili Powder": "to taste",
    "Salt and Pepper": "to taste",
    "Ginger": 2,
    "Lime Juice": 3,
    "Soy Sauce": 2,
    "Tahini or Peanut Butter": 4,
    "Garlic Powder": 1,
    "Balsamic Vinegar": 1,
    "Cumin": 0.5,
    "Chili or Cayenne Pepper": "to taste"
  },
  "Oils":

In [94]:
import json
outputShoppingListFile = './outputShoppingList/w52_plan_Ingredients_03_ShoppingList_03.json'
json_Shopping = json.loads(outputShoppingListFile)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [93]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_json(outputShoppingListFile)
df.head()

,Vegetables,Proteins,Grains,Dairy,Canned Goods,Spices and Condiments,Oils,Others
Onions,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bell Peppers,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Garlic Cloves,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Spring Onions,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carrots,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
###################################
###################################
###################################
####### CATEGORIES 01 #############
######## 5 examples ###############
###################################
# INPUT: Extrapolation, 10 word max summary of texts.
# OUTPUT: Categories, categorization of the extrapolated texts

# Setup new file of categories
output_Dir = "./outputOnlyCategories/"
nameOutput_File = "identifyOnlyCategories_"
output_File = output_directories(output_Dir,nameOutput_File)
outputProgress_Dir = "./progressOnlyCategories/"
outputProgress_File = output_directories(outputProgress_Dir,"progressIdentifyOnlyCategories_")

# load extrapolation file
pathFile = "./outputExtrapolation/"
nameFile = "extrapol*.txt"
list_of_files = glob.glob(pathFile + nameFile) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
content = read_file(latest_file)

####################################
####################################
# Prompt for Categories
####################################
####################################

prompt01 = f"""

Your task is to extract the topics discussed in a sequence of sentences delimited by <>. 
Each sentence is represented by a number. \

To perform the task follow these steps: \
- work out the topics discussed in the sentences. \
- reflect if you can find better topics to represent the sentences. \
- explain each topic in a concise way. \

Sentences: <{content}>
"""


temperature = 0
model = 'gpt-4'
response01 = get_completion(prompt01,model,temperature)
response02 = get_completion(prompt01,model,temperature)
response03 = get_completion(prompt01,model,temperature)

prompt02 = f"""

Follow these steps: \
- work out the topics delimited by "" best represent the sentences delimited by <>.
- each sentence is represented by a number. \
- each topic is followed by an explanation. \
- use the explanation of the topic to find out which topics best represent the sentences. \
- explain each resulting topic in a concise way. \
- output the response as a python dictionary that contains the following 
key: topic name, and values: explanation. \

Sentences: <{content}>\

Topics: " \

{response01} \

{response02} \

{response02} \

"
"""

response = get_completion(prompt02,model,temperature)

print(response)

#######################################
#######################################
# Saving results
    
f = open(output_File, "w")
f.write(response)
f.close()
f = open(outputProgress_File, "w")
f.write("temperature: ")
f.write(str(temperature))
f.write(", model: ")
f.write(model)
f.write("\n")
f.write("prompt01: \n")
f.write(prompt01)
f.write("prompt02: \n")
f.write(prompt02)
f.write("\n")
f.write("response: \n")
f.write(response)
f.close()


In [16]:
###################################
###################################
###################################
####### CATEGORIES ################
###################################
###################################
# INPUT: Extrapolation, 10 word max summary of texts.
# OUTPUT: Categories, categorization of the extrapolated texts

# Setup new file of categories
output_Dir = "./outputOnlyCategories/"
nameOutput_File = "identifyOnlyCategories_"
output_File = output_directories(output_Dir,nameOutput_File)
outputProgress_Dir = "./progressOnlyCategories/"
outputProgress_File = output_directories(outputProgress_Dir,"progressIdentifyOnlyCategories_")

# load extrapolation file
pathFile = "./outputExtrapolation/"
nameFile = "extrapol*.txt"
list_of_files = glob.glob(pathFile + nameFile) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
content = read_file(latest_file)

####################################
####################################
# Prompt for Categories
####################################
####################################

prompt = f"""
Your task is to extract the topics discussed in a sequence of sentences delimited by <>. 
Each sentence is represented by a number. \

To perform the task follow these steps: \
- work out the topics discussed in the sentences. \
- reflect if you can find better topics to represent the sentences. \
- explain each topic in a concise way. \
- output the response as a python dictionary that contains the following 
key: topic name, and values: explanation. \

Sentences: <{content}>
"""


temperature = 0
model = 'gpt-4'
response = get_completion(prompt,model,temperature)

print(response)

#######################################
#######################################
# Saving results
    
f = open(output_File, "w")
f.write(response)
f.close()
f = open(outputProgress_File, "w")
f.write("temperature: ")
f.write(str(temperature))
f.write(", model: ")
f.write(model)
f.write("\n")
f.write("prompt: \n")
f.write(prompt)
f.write("\n")
f.write("response: \n")
f.write(response)
f.close()


{
  "Education System": "Covers the overall quality, funding, and curriculum of schools.",
  "Teaching Methods": "Refers to the approaches and techniques used in classrooms, including engagement and flexibility.",
  "Parental Feedback": "Represents the opinions and satisfaction of parents with the education system.",
  "School Staff": "Addresses the stability, motivation, and qualifications of teachers and other school personnel.",
  "Neighborhood Factors": "Discusses the impact of local population and neighborhood on school quality."
}


In [17]:
###################################
###################################
###################################
####### PAIRING  ##################
###### EXTRAPOLATION ##############
####### CATEGORIES ################
###################################
###################################
# INPUT: Extrapolation and Categories.
# OUTPUT: Index of which extrapolation best fits each category.

# Setup new file of categories
output_Dir = "./outputPairing/"
nameOutput_File = "pairingExtrapolCategory_"
output_File = output_directories(output_Dir,nameOutput_File)
outputProgress_Dir = "./progressPairingExtrapolCategory/"
outputProgress_File = output_directories(outputProgress_Dir,"progressPairingExtrapolCategory_")

# load extrapolation file
pathFile = "./outputExtrapolation/"
nameFile = "extrapol*.txt"
list_of_files = glob.glob(pathFile + nameFile) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
contentExtrapolations = read_file(latest_file)

# load categories file
pathFile = "./outputOnlyCategories/"
nameFile = "*Only*.txt"
list_of_files = glob.glob(pathFile + nameFile) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
contentCategories = read_file(latest_file)

####################################
####################################
# Prompt for pairing 
#  extrapolations 
#       with
#    categories
####################################
####################################

prompt = f"""
Your task is to associate the sentences delimited by <> to a category from the python dictionary delimited by "". \
Associate the sentences to a category by using its explanation value of the python dictionary. \
Each sentence is represented by a number. \

To perform the task follow these steps: \
- use the provided explanations to cluster the sentences. \
- reflect on how the explanations explain the sentences. \
- use only the provided categories. \
- associate all the sentences. \
- one sentence can be associated with more than one category. \
- output the response as a python dictionary that contains the following 
key: category name, and value: sentence number. \

Sentences: <{contentExtrapolations}> \

Categories: "{contentCategories}"
"""


temperature = 0
model = 'gpt-4'
response = get_completion(prompt,model,temperature)

print(response)

#######################################
#######################################
# Saving results
    
f = open(output_File, "w")
f.write(response)
f.close()
f = open(outputProgress_File, "w")
f.write("temperature: ")
f.write(str(temperature))
f.write(", model: ")
f.write(model)
f.write("\n")
f.write("prompt: \n")
f.write(prompt)
f.write("\n")
f.write("response: \n")
f.write(response)
f.close()



{
  "Education System": [1, 3, 4, 5, 6, 10],
  "Teaching Methods": [1, 6, 10],
  "Parental Feedback": [2, 5],
  "School Staff": [7, 8, 9, 11],
  "Neighborhood Factors": [4]
}
